In [1]:
import numpy as np
import pandas as pd
import tls_client
from bs4 import BeautifulSoup
from unidecode import unidecode
from betting_functions import get_url_soup
import time

In [2]:
requests = tls_client.Session(
    client_identifier="chrome112",
)
year = '2024'

In [3]:
soup = get_url_soup(f'https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html')

# Find all <td> elements with the 'data-append-csv' attribute
td_elements = soup.find_all('td', {'data-append-csv': True})

# Create empty lists to store the extracted data
player_codes = []

# Loop through the <td> elements
for td_element in td_elements:
    data_append_csv = td_element['data-append-csv']
    player_name = td_element.find('a').text
    player_codes.append([player_name,data_append_csv])

# Print the extracted data
print(player_codes[0:10])
print(len(player_codes))


[['Precious Achiuwa', 'achiupr01'], ['Bam Adebayo', 'adebaba01'], ['Ochai Agbaji', 'agbajoc01'], ['Santi Aldama', 'aldamsa01'], ['Nickeil Alexander-Walker', 'alexani01'], ['Grayson Allen', 'allengr01'], ['Jarrett Allen', 'allenja01'], ['Kyle Anderson', 'anderky01'], ['Giannis Antetokounmpo', 'antetgi01'], ['Thanasis Antetokounmpo', 'antetth01']]
448


In [4]:
rows = soup.find_all('tr', class_=['full_table', 'italic_text partial_table'])

# Create a list to store the 'pos' values
positions = []

# Loop through the rows and extract the 'pos' value for each row
for row in rows:
    pos_element = row.find('td', {'data-stat': 'pos'})
    if pos_element:
        position = pos_element.text
        positions.append(position)
print(len(player_codes),len(positions))
code_df = pd.DataFrame(player_codes,columns=['name','code'])
code_df['pos'] = positions
code_df.drop_duplicates(inplace=True)
code_df = code_df.reset_index(drop=True)

448 448


In [5]:
code_df.iloc[0:10]

,name,code,pos
0,Precious Achiuwa,achiupr01,C
1,Bam Adebayo,adebaba01,C
2,Ochai Agbaji,agbajoc01,SG
3,Santi Aldama,aldamsa01,PF
4,Nickeil Alexander-Walker,alexani01,SG
5,Grayson Allen,allengr01,SG
6,Jarrett Allen,allenja01,C
7,Kyle Anderson,anderky01,PF
8,Giannis Antetokounmpo,antetgi01,PF
9,Thanasis Antetokounmpo,antetth01,PF


In [6]:
code = player_codes[0][1]
base = f'https://www.basketball-reference.com/players/c/{code}/gamelog/'
# NBA season we will be analyzing
url = base+year
print(url)
# this is the HTML from the given URL
response1 = requests.get(url)
soup = BeautifulSoup(response1.content)

https://www.basketball-reference.com/players/c/achiupr01/gamelog/2024


In [7]:
def get_headers(soup):
    i = 0
    while i < 40:
        headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
        i = i +1
        if headers:
            idx = i
            i = 41
    return headers[1:]

In [8]:
def get_stats_df(soup,headers,player):

    rows = soup.findAll('tr')
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    player_stats = [x for x in player_stats if len(x) > 4]
    stats = pd.DataFrame(player_stats, columns = headers)
    stats.insert(0, 'player', player)
    stats.index = range(len(stats))
    return stats

#soup = get_url_soup(url)
#headers = get_headers(soup)
#get_stats_df(soup,headers,code_df['code'].iloc[0])

In [9]:
def get_stats(num,df):
    for person in range(num):
        errors = []
        try:
            player_id = df['code'].iloc[person]
            player_name = df['name'].iloc[person]
            base = f'https://www.basketball-reference.com/players/c/{player_id}/gamelog/'
            url = base+year
            time.sleep(2.95)
            soup = get_url_soup(url)
            
            headers = get_headers(soup)
            stats = get_stats_df(soup,headers,player_name)
            stats['pos'] = df['pos'].iloc[person]
            stats['season'] = year
            if person == 0:
                data_core = stats
            else:
                data_core = pd.concat([data_core,stats])
            if person % 5 == 0:
                print(url)
                print(person,player_name)
        except: 
            errors.append(player_id)
    print(errors)
    return data_core


In [10]:
print(len(code_df))
test = get_stats(len(code_df),code_df)

446
https://www.basketball-reference.com/players/c/achiupr01/gamelog/2024
0 Precious Achiuwa
https://www.basketball-reference.com/players/c/adebaba01/gamelog/2024
1 Bam Adebayo
https://www.basketball-reference.com/players/c/agbajoc01/gamelog/2024
2 Ochai Agbaji
https://www.basketball-reference.com/players/c/aldamsa01/gamelog/2024
3 Santi Aldama
https://www.basketball-reference.com/players/c/alexani01/gamelog/2024
4 Nickeil Alexander-Walker
https://www.basketball-reference.com/players/c/allengr01/gamelog/2024
5 Grayson Allen
https://www.basketball-reference.com/players/c/allenja01/gamelog/2024
6 Jarrett Allen
https://www.basketball-reference.com/players/c/anderky01/gamelog/2024
7 Kyle Anderson
https://www.basketball-reference.com/players/c/antetgi01/gamelog/2024
8 Giannis Antetokounmpo
https://www.basketball-reference.com/players/c/antetth01/gamelog/2024
9 Thanasis Antetokounmpo
https://www.basketball-reference.com/players/c/anthoco01/gamelog/2024
10 Cole Anthony
https://www.basketball-

In [11]:
with pd.option_context('display.max_rows', None,
                       'display.precision', 3,
                       ):
    display(test.sample(5))

,player,G,Date,Age,Tm,,Opp,,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,pos,season
4,Jabari Walker,5,2023-11-01,21-094,POR,@,DET,W (+9),0,9:56,...,0,0,0,1,0,1,-1.2,-6,SF,2024
3,Jordan Hawkins,4,2023-11-01,21-186,NOP,@,OKC,W (+4),1,37:18,...,4,2,0,1,2,10,4.7,+17,SG,2024
4,Cam Reddish,5,2023-11-01,24-061,LAL,,LAC,W (+5),1,36:38,...,4,3,1,0,5,8,9.2,-4,SF,2024
6,Moses Moody,7,2023-11-05,21-158,GSW,@,CLE,L (-11),0,14:18,...,0,1,0,0,1,3,2.8,-1,SG,2024
1,Sasha Vezenkov,2,2023-10-27,28-082,SAC,,GSW,L (-8),0,12:16,...,0,0,0,0,0,10,7.7,+1,PF,2024


In [12]:
save = test.copy(deep=True)
save.tail(6)

,player,G,Date,Age,Tm,,Opp,,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,pos,season
5,Cody Zeller,,2023-11-04,31-030,NOP,,ATL,L (-18),Did Not Play,None,...,None,None,None,None,None,None,None,None,C,2024
0,Ivica Zubac,1,2023-10-25,26-221,LAC,,POR,W (+12),1,25:36,...,0,0,4,2,3,20,21.0,+25,C,2024
1,Ivica Zubac,2,2023-10-27,26-223,LAC,@,UTA,L (-2),1,19:26,...,0,0,2,1,2,6,2.4,+1,C,2024
2,Ivica Zubac,3,2023-10-29,26-225,LAC,,SAS,W (+40),1,22:29,...,0,0,1,1,4,16,14.0,+23,C,2024
3,Ivica Zubac,4,2023-10-31,26-227,LAC,,ORL,W (+16),1,22:41,...,1,0,1,1,1,9,8.1,+19,C,2024
4,Ivica Zubac,5,2023-11-01,26-228,LAC,@,LAL,L (-5),1,29:06,...,0,0,0,1,5,7,2.7,-6,C,2024


In [13]:
test['Date'] = pd.to_datetime(test['Date'])
data = test
for i in range(len(data)):
    data.iloc[i,0] = unidecode(data.iloc[i,0]).replace('_',' ')
    #remove special characters

In [14]:
data.head(5)

,player,G,Date,Age,Tm,,Opp,,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,pos,season
0,Precious Achiuwa,1,2023-10-25,24-036,TOR,,MIN,W (+3),0,24:17,...,0,0,0,2,1,8,4.5,-5,C,2024
1,Precious Achiuwa,2,2023-10-27,24-038,TOR,@,CHI,L (-1),0,22:38,...,1,0,0,3,6,8,4.1,-22,C,2024
2,Precious Achiuwa,3,2023-10-28,24-039,TOR,,PHI,L (-7),0,18:15,...,3,0,1,1,1,6,4.9,+5,C,2024
3,Precious Achiuwa,,2023-10-30,24-041,TOR,,POR,L (-8),Inactive,None,...,None,None,None,None,None,None,None,None,C,2024
4,Precious Achiuwa,,2023-11-01,24-043,TOR,,MIL,W (+19),Inactive,None,...,None,None,None,None,None,None,None,None,C,2024


In [15]:
data = data.reset_index(drop=True)

In [16]:
data['pos'].value_counts()

pos
SG       676
SF       563
PF       524
C        471
PG       449
PF-SF     10
Name: count, dtype: int64

In [17]:
data.dropna(inplace=True)
minutes =  data['MP'].to_list()

for i in range(len(minutes)):
    new = minutes[i].split(':')
    res = float(new[0])+(float(new[1])/60)
    minutes[i] = res
data['MP'] = minutes

In [18]:
data.columns

Index(['player', 'G', 'Date', 'Age', 'Tm', ' ', 'Opp', ' ', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'pos',
       'season'],
      dtype='object')

In [19]:
data.columns = ['player', 'G', 'date', 'age', 'team', 'H/A', 'Opp', 'W/L', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'pos','season']
home = data['H/A'].values
print(home[0:4])
for i in range(len(home)):
    if home[i] == '@':
        home[i] = 0
    else:
        home[i] = 1
home[0:10]


['' '@' '' '']


array([1, 0, 1, 1, 0, 0, 1, 1, 1, 1], dtype=object)

In [20]:
data

,player,G,date,age,team,H/A,Opp,W/L,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,pos,season
0,Precious Achiuwa,1,2023-10-25,24-036,TOR,1,MIN,W (+3),0,24.283333,...,0,0,0,2,1,8,4.5,-5,C,2024
1,Precious Achiuwa,2,2023-10-27,24-038,TOR,0,CHI,L (-1),0,22.633333,...,1,0,0,3,6,8,4.1,-22,C,2024
2,Precious Achiuwa,3,2023-10-28,24-039,TOR,1,PHI,L (-7),0,18.250000,...,3,0,1,1,1,6,4.9,+5,C,2024
7,Bam Adebayo,1,2023-10-25,26-099,MIA,1,DET,W (+1),1,32.133333,...,3,0,2,2,3,22,18.4,-4,C,2024
8,Bam Adebayo,2,2023-10-27,26-101,MIA,0,BOS,L (-8),1,35.016667,...,2,1,2,3,1,27,17.8,0,C,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2688,Ivica Zubac,1,2023-10-25,26-221,LAC,1,POR,W (+12),1,25.600000,...,0,0,4,2,3,20,21.0,+25,C,2024
2689,Ivica Zubac,2,2023-10-27,26-223,LAC,0,UTA,L (-2),1,19.433333,...,0,0,2,1,2,6,2.4,+1,C,2024
2690,Ivica Zubac,3,2023-10-29,26-225,LAC,1,SAS,W (+40),1,22.483333,...,0,0,1,1,4,16,14.0,+23,C,2024
2691,Ivica Zubac,4,2023-10-31,26-227,LAC,1,ORL,W (+16),1,22.683333,...,1,0,1,1,1,9,8.1,+19,C,2024


In [21]:
data['H/A'] = home
data = data.replace('CHO','CHA')
data = data.replace('PHO','PHX')
data= data.replace('BRK','BKN')

data['W/L'] = data['W/L'].str.extract(r"\(([-+]?\d+)\)").astype(int)
data

,player,G,date,age,team,H/A,Opp,W/L,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,pos,season
0,Precious Achiuwa,1,2023-10-25,24-036,TOR,1,MIN,3,0,24.283333,...,0,0,0,2,1,8,4.5,-5,C,2024
1,Precious Achiuwa,2,2023-10-27,24-038,TOR,0,CHI,-1,0,22.633333,...,1,0,0,3,6,8,4.1,-22,C,2024
2,Precious Achiuwa,3,2023-10-28,24-039,TOR,1,PHI,-7,0,18.250000,...,3,0,1,1,1,6,4.9,+5,C,2024
7,Bam Adebayo,1,2023-10-25,26-099,MIA,1,DET,1,1,32.133333,...,3,0,2,2,3,22,18.4,-4,C,2024
8,Bam Adebayo,2,2023-10-27,26-101,MIA,0,BOS,-8,1,35.016667,...,2,1,2,3,1,27,17.8,0,C,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2688,Ivica Zubac,1,2023-10-25,26-221,LAC,1,POR,12,1,25.600000,...,0,0,4,2,3,20,21.0,+25,C,2024
2689,Ivica Zubac,2,2023-10-27,26-223,LAC,0,UTA,-2,1,19.433333,...,0,0,2,1,2,6,2.4,+1,C,2024
2690,Ivica Zubac,3,2023-10-29,26-225,LAC,1,SAS,40,1,22.483333,...,0,0,1,1,4,16,14.0,+23,C,2024
2691,Ivica Zubac,4,2023-10-31,26-227,LAC,1,ORL,16,1,22.683333,...,1,0,1,1,1,9,8.1,+19,C,2024


In [22]:
data['date'].max()

Timestamp('2023-11-05 00:00:00')

In [25]:
KM_vals = pd.read_csv('./reference_data/KM_vals.csv',index_col=0)

In [26]:
KM_dict = dict(zip(KM_vals.Player,KM_vals.KM))

In [27]:
temp = []
for p in data.player.values:
    if p[0] in KM_dict:
        temp.append(KM_dict[p[0]])
    else:
        temp.append(15)
data['KM'] = temp

In [29]:
data.to_csv(f'/GIthub/NBA_betting_model/game_logs/data_{year}.csv',index=False)